#### Project 3: The worm algorithm for the 6-vertex model (Monte Carlo)

Implement the worm algorithm for the 6-vertex (ice-type) model (https://en.wikipedia.org/wiki/Ice-type_model) at $T = \infty$  
(where all allowed configurations are equally likely).

Calculate the exponent $a$ of the correlation function $\langle s_0 s_r \rangle \propto \frac{1}{r^a}$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [232]:
class six_vertex_model(object):
    """Ice-type (6-vertex) model class.
    
    Attributes
    ----------
    Lx : int
        Lattice X size.
    Ly : int
        Lattice Y size.
    N : int
        Number of vertices.
    lattice : list
        Horizontal (hor) & vertical (ver) sublattices.
    """
    
    def __init__(self, Lx, Ly, hor=[], ver=[]): 
        '''hor, ver: initial lattice configuration
        '''
        self.Lx = Lx
        self.Ly = Ly
        self.N  = Lx * Ly;
        
        # periodic BC: hor[_][0] = hor[_][Lx+1] && ver[_][0] = ver[_][Ly+1]
        if hor == [] or ver == []:
            hor = np.ones((Ly, Lx), dtype=np.int8) 
            ver = np.ones((Lx, Ly), dtype=np.int8)
            hor[1::2] *= -1
            ver[1::2] *= -1
        assert Ly == hor.shape[0] and Lx == ver.shape[0]
        
        self.lattice = np.array([hor, ver])
        # print(self.lattice[0].shape, self.lattice[1].shape) # (Ly, Lx), (Lx, Ly)
        # self.n = np.array(['...']*6) # dummy: how many in each of the 6 config types
    
    def plot(self):
        print('Horizontal spins:')
        print(self.lattice[0], '\n')
        print('Vertical spins:')
        print(self.lattice[1].transpose())
    
    def energy(self, energies=[1.]*6):
        '''dummy: energies : list with energies of the 6 configurations.
        '''
        return self.N # T->Inf: all equally likely
        # return np.sum(self.n * np.array(energies))

    def flip_spin(self, x, y, flip):
        '''Create 2 defects by flipping 1 spin.
        x, y : vertex coord
        flip : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        if flip == 0:   # left
            self.lattice[0, x, y] *= -1
        elif flip == 1: # right
            self.lattice[0, x, (y+1) % self.Lx] *= -1
        elif flip == 2: # up
            self.lattice[1, y, x] *= -1
        else:           # down
            self.lattice[1, y, (x+1) % self.Ly] *= -1
    
    def get_other_defect(x, y, flip):
        '''Return coord of neighbour defect
        x, y : vertex coord
        flip : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        if flip == 0:   # left
            return (x, (y-1) % self.Lx)
        elif flip == 1: # right
            return (x, (y+1) % self.Lx)
        elif flip == 2: # up
            return ((x-1) % self.Ly, y)
        else:           # down
            return ((x+1) % self.Ly, y)

    def get_vertex_in_out(self, x, y):
        '''Return in_or_out: [spin_left, spin_right, spin_up, spin_down]
        relative to vertex (x, y).
        In  :  1
        Out : -1
        '''
        spin_left  = self.lattice[0, x, y]
        spin_right = self.lattice[0, x, (y+1) % self.Lx] * -1
        spin_up    = self.lattice[1, y, x] * -1
        spin_down  = self.lattice[1, y, (x+1) % self.Ly]
        return np.array[(spin_left, spin_right, spin_up, spin_down)]

    def flip_next(self, x, y, flip):
        '''Propagate defect by flipping randomly one of the next vertex's spins,
        subject to the ice rule (2-in, 2-out).
        '''
        prob = np.random.random() # 50/50 chance
        in_or_out = self.get_vertex_in_out(x, y)
        
        flipped = in_or_out[flip]
        # possible = np.arange(4) == 
        
        
        return x, y, flip
    
        # if self.lattice[orient, x, (y+1) % self.Lx] == self.lattice[orient, x, y]:
        #     if prob < 0.5:
        #         self.flip_spin(not orient, y, x)
        #     else:
        #         self.flip_spin(not orient, y, (x+1) % self.Ly)
        # else:
        #     if prob < 0.5:
        #         self.flip_spin(orient, x, (y+1) % self.Lx)
        #     else:
        #         if self.lattice[not orient, y, x] != self.lattice[orient, x, y]:
        #             self.flip_spin(not orient, y, x)
        #         else:
        #             self.flip_spin(not orient, y, (x+1) % self.Ly)
    
    def propagate_2_defects(self):
        '''Flip 1 spin, create 2 defective vertices & propagate one randomly until they
        meet & annihilate each other.
        '''
        x = np.random.randint(self.Lx) # vertex x coord
        y = np.random.randint(self.Ly) # vertex y coord
        flip = np.random.randint(4) # 0: left, 1: right, 2: up, 3: down
        print(x, y, flip)
        
        self.flip_spin(x, y, flip)
        end_vertex = self.get_other_defect(x, y, flip) # end vertex coord
        
        propagate = True
        while propagate:
            x, y, flip = self.flip_next(x, y, flip)
            if (x, y) == end_vertex: # defects met & annhilated
                propagate = False

L = 3
icetype = six_vertex_model(L, L)
# icetype.energy([1, 1, 1, 1, 1, 1])
# icetype.plot()

In [230]:
icetype.flip_spin(1, 2, 3)
# icetype.propagate_2_defects()
icetype.plot()
# icetype.flip_next()

Horizontal spins:
[[ 1 -1  1]
 [-1 -1 -1]
 [ 1  1  1]] 

Vertical spins:
[[ 1 -1  1]
 [ 1 -1  1]
 [ 1 -1 -1]]


In [ ]:
# 1) thermalize
# 2) measure -> correlations
# 3) update -> (2) x10000